In [1]:
import argparse
import datetime
import numpy as np

from tensorflow.keras.callbacks import ModelCheckpoint, History, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.layers import BatchNormalization, Input, Reshape, Dense, Conv1D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from timeit import default_timer as timer
from src.train.utils import build_numpy, build_generator, fetch_file_list_pn, BATCH_SIZE, build_numpy, \
    fetch_file_list


X_COLUMNS = slice(7, 76)
Y_COLUMNS = 0
BATCH_SIZE = 128


EPOCHS = 20
np.random.seed(100)


TRAINING_DATA_DIR = '/Users/lucindazhao/strava/ml-local/trainData/set1/'
VALIDATION_DATA_DIR = '/Users/lucindazhao/strava/ml-local/validationData/set1/'
MODEL_CHECKPOINT = '/Users/lucindazhao/strava/ml-local/snapshots/'
LOG_DIR = '/Users/lucindazhao/strava/ml-local/logs/'


IS_CSV = False
TRAIN_PORTION = 0.3
VALIDATION_PORTION = 1.1
# number of samples. A tiny number of samples for fast test
# For real training should reflect number of samples in whole training set
NUM_TRAIN_SAMPLES = int(9625835 * TRAIN_PORTION)
# NUM_TRAIN_SAMPLES = int(9625835.0 /30)
NUM_VALI_SAMPLES = int(1273483.0 / 6)

def build_fc_network(l2=1e-3):
    # build fully-connected network
    l = Input(shape=(69,))
    inputs = l
    l = Dense(128, activation="relu", name='dense1', kernel_regularizer=regularizers.l2(l2))(l)
    l = BatchNormalization()(l)
    l = Dropout(0.3)(l)
    l = Dense(128, activation="relu", name='dense2', kernel_regularizer=regularizers.l2(l2))(l)
    l = BatchNormalization()(l)
    l = Dropout(0.3)(l)
    l = Dense(64, activation="relu", name='dense3', kernel_regularizer=regularizers.l2(l2))(l)
    l = BatchNormalization()(l)
    l = Dense(64, activation="relu", name='dense4', kernel_regularizer=regularizers.l2(l2))(l)
    l = BatchNormalization()(l)
    l = Dense(64, activation="relu", name='dense5', kernel_regularizer=regularizers.l2(l2))(l)
    l = BatchNormalization()(l)
    l = Dropout(0.4)(l)
    l = Dense(64, activation="relu", name='dense6', kernel_regularizer=regularizers.l2(l2))(l)
    l = BatchNormalization()(l)
    l = Dense(64, activation="relu", name='dense7', kernel_regularizer=regularizers.l2(l2))(l)
    l = BatchNormalization()(l)
    l = Dense(64, activation="relu", name='dense8', kernel_regularizer=regularizers.l2(l2))(l)
    l = BatchNormalization()(l)
    l = Dense(64, activation="relu", name='dense9')(l)
    l = BatchNormalization()(l)
    l = Dropout(0.3)(l)
    l = Dense(32, activation="relu", name='dense10')(l)
    l = BatchNormalization()(l)
    l = Dense(32, activation="relu", name='dense11')(l)
    l = BatchNormalization()(l)
    l = Dense(32, activation="relu", name='dense12')(l)
    l = BatchNormalization()(l)
    l = Dense(32, activation="relu", name='dense13')(l)
    l = BatchNormalization()(l)
    l = Dropout(0.3)(l)
    l = Dense(2, activation="softmax", name='main_output')(l)
    # for lr in np.logspace(start=1, stop=-4, num=5, base=10.0):
    m_fc = Model(inputs=inputs, outputs=l)
    print(m_fc.summary())
    return 'fc_noWeight_13layer_l2={}_'.format(l2), m_fc


def build_cnn_network():
    l = Input(shape=(69,))
    inputs = l
    l = Reshape((69, 1))(l)
    for i in range(0, 4):
        #  Output_size = input_size + 2 * padding_size-(filter_size-1)
        l = Conv1D(16, 3, padding="same", activation="relu", name='conv3' + str(i))(l)
        l = BatchNormalization(name='batch3_' + str(i))(l)
        l = Dropout(0.5)(l)
    # 69 -4 * 10 --> 29
    for i in range(0, 10):
        l = Conv1D(32, 5, padding="valid", activation="relu", strides=[1], name='conv5_{}'.format(i))(l)
        l = BatchNormalization(name='batch7_' + str(i))(l)
        l = Dropout(0.5)(l)
    
    l = Conv1D(64, 3, padding="same", activation="relu", name='conv3_last5')(l)
    l = BatchNormalization()(l)
    
    l = Conv1D(32, 3, padding="same", activation="relu", name='conv3_last4')(l)
    l = BatchNormalization()(l)
    l = Dropout(0.3)(l)

    
    l = Conv1D(16, 3, padding="same", activation="relu", name='conv3_last3')(l)
    l = BatchNormalization()(l)
    l = Dropout(0.3)(l)

    l = Conv1D(8, 3, padding="same", activation="relu", name='conv3_last2')(l)
    l = BatchNormalization()(l)

    l = Conv1D(4, 3, padding="same", activation="relu", name='conv3_last1')(l)
    l = BatchNormalization()(l)
    l = Dropout(0.3)(l)



    l = Reshape((4*29,))(l)
    l = Dense(16, activation="relu", name='fc')(l)
    l = Dense(2, activation="softmax", name='main_output')(l)

    m_cnn = Model(inputs=inputs, outputs=l)
    print(m_cnn.summary())
    return 'cnn_noWeight_20layer', m_cnn



Using TensorFlow backend.


# Train Mini Model First

In [2]:
train_file_list = fetch_file_list(data_dir=TRAINING_DATA_DIR, portion=TRAIN_PORTION)
tg = build_numpy(file_list=train_file_list, num_samples=None, xcolumns=X_COLUMNS, ycolumns=Y_COLUMNS, ytx=None,
                 skip_header=1, shuffle=False, is_csv=IS_CSV)
val_file_list = fetch_file_list(data_dir=VALIDATION_DATA_DIR, portion=VALIDATION_PORTION)
vg = build_numpy(file_list=val_file_list, num_samples=None, xcolumns=X_COLUMNS, ycolumns=Y_COLUMNS, ytx=None,
                 skip_header=1, shuffle=False, is_csv=IS_CSV)



Number of Files: 1


ValueError: need at least one array to concatenate

In [3]:
for l2 in [1e-3]:
    model_pair = build_cnn_network()
    key = model_pair[0]
    model = model_pair[1]
    history = History()
    log_dir = LOG_DIR + key + '/' + 'lr=' + str(lr) + '_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    file_path = MODEL_CHECKPOINT + key +  'lr=' + str(lr) + '_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".h5"
    tensor_board = TensorBoard(log_dir, profile_batch=0)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                        patience=2, min_lr=1e-5)
    checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=5)

    callbacks_list = [history, tensor_board, reduce_lr, checkpoint]


    print(log_dir)
    print(file_path)
    adam_wn = Adam(lr=lr)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=adam_wn, metrics=['accuracy'])
    model.fit(x=tg[0], y=tg[1], validation_data=vg, batch_size=BATCH_SIZE,
                  epochs=EPOCHS, verbose=1, shuffle=False,
                  callbacks=callbacks_list, class_weight={0: 1.0, 1: 1.0})

    model.save(file_path)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 69)]              0         
_________________________________________________________________
reshape (Reshape)            (None, 69, 1)             0         
_________________________________________________________________
conv30 (Conv1D)              (None, 69, 16)            64        
_________________________________________________________________
batch3_0 (BatchNormalization (None, 69, 16)            64        
_________________________________________________________________
dropout (Dropout)            (None, 69, 16)            0         
_________________________________________________________________
conv31 (Conv1D)              (None, 69, 16)            784       
_________________________________________________________________
batch3_1 (BatchNormalization (None, 69, 16)            64    

NameError: name 'lr' is not defined

# Train Serious Model

In [3]:
start = timer()
train_file_list = fetch_file_list(data_dir=TRAINING_DATA_DIR, portion=TRAIN_PORTION)
tg = build_generator(file_list=train_file_list, xcolumns=X_COLUMNS, ycolumns=Y_COLUMNS, ytx=None, is_csv=IS_CSV)
v_file_list = fetch_file_list(data_dir=VALIDATION_DATA_DIR, portion=VALIDATION_PORTION)
vg = build_generator(file_list=v_file_list, xcolumns=X_COLUMNS, ycolumns=Y_COLUMNS, ytx=None, is_csv=IS_CSV)
end = timer()
print("{} min taken for generating input".format((end - start) / 60.0))

Number of Files: 3
Number of Files: 1
5.2575216007729374e-05 min taken for generating input


In [6]:
history = History()
log_dir = LOG_DIR + key + '/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_path = MODEL_CHECKPOINT + key + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".h5"

tensor_board = TensorBoard(log_dir, histogram_freq=5)  # profile_batch=0
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                            patience=2, min_lr=1e-5)
callbacks_list = [history, tensor_board, reduce_lr]

adam_wn = Adam(lr=LR)
model.compile(loss="sparse_categorical_crossentropy", optimizer=adam_wn, metrics=['accuracy'])

model.fit_generator(generator=tg, steps_per_epoch=int(NUM_TRAIN_SAMPLES/BATCH_SIZE),
                    epochs=EPOCHS, verbose=1, shuffle=True,
                    use_multiprocessing=True, workers=10,
                    callbacks=callbacks_list,
                    max_queue_size=len(train_file_list),
                    validation_data=vg, validation_steps=int(NUM_VALI_SAMPLES/BATCH_SIZE),
                    class_weight={0: 1.0, 1: 10.0})

model.save(file_path)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 7520 steps, validate for 2984 steps
Epoch 1/10
1057/7520 [===>..........................] - ETA: 4:37 - loss: 0.9010 - accuracy: 0.7090

Process Keras_worker_ForkPoolWorker-29:
Process Keras_worker_ForkPoolWorker-27:
Process Keras_worker_ForkPoolWorker-23:
Process Keras_worker_ForkPoolWorker-28:
Process Keras_worker_ForkPoolWorker-22:
Process Keras_worker_ForkPoolWorker-25:
Process Keras_worker_ForkPoolWorker-21:
Process Keras_worker_ForkPoolWorker-24:
Process Keras_worker_ForkPoolWorker-26:
Process Keras_worker_ForkPoolWorker-30:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.p

  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInt

1058/7520 [===>..........................] - ETA: 4:37 - loss: 0.9010 - accuracy: 0.7090

KeyboardInterrupt
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

Process Keras_worker_ForkPoolWorker-32:
Process Keras_worker_ForkPoolWorker-37:
Process Keras_worker_ForkPoolWorker-36:
Process Keras_worker_ForkPoolWorker-33:
Process Keras_worker_ForkPoolWorker-38:
Process Keras_worker_ForkPoolWorker-35:
Process Keras_worker_ForkPoolWorker-31:
Process Keras_worker_ForkPoolWorker-34:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Ver

KeyboardInterrupt


# Load pre-trained model and continue training

In [196]:
file_path = '/Users/lucindazhao/strava/ml-local/snapshots/cnn_noWeight_lr=0.001_20200218-224657.h5'
model_path = file_path.split('.h5')[0]
print(model_path)
new_path = model_path + '_resume_{}.h5'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
log_dir = LOG_DIR + model_path.split('/')[-1] \
          + '_resume_{}.h5'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
print(new_path)
print(model_path.split('/')[-1])
print(log_dir)

model = load_model(model_path + '.h5')
print(model.summary())

/Users/lucindazhao/strava/ml-local/snapshots/cnn_noWeight_lr=0.001_20200218-224657
/Users/lucindazhao/strava/ml-local/snapshots/cnn_noWeight_lr=0.001_20200218-224657_resume_20200219-222919.h5
cnn_noWeight_lr=0.001_20200218-224657
/Users/lucindazhao/strava/ml-local/logs/cnn_noWeight_lr=0.001_20200218-224657_resume_20200219-222919.h5
Model: "model_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_48 (InputLayer)        [(None, 69)]              0         
_________________________________________________________________
reshape_22 (Reshape)         (None, 69, 1)             0         
_________________________________________________________________
conv0 (Conv1D)               (None, 69, 32)            128       
_________________________________________________________________
batch0 (BatchNormalization)  (None, 69, 32)            128       
______________________________________________________

In [177]:
# load and resume

# start = timer()
# train_file_list = fetch_file_list(data_dir=TRAINING_DATA_DIR, portion=TRAIN_PORTION)
# tg = build_numpy(file_list=train_file_list, num_samples=None, xcolumns=X_COLUMNS, ycolumns=Y_COLUMNS, ytx=None,
#                  skip_header=1, shuffle=False, is_csv=IS_CSV)
# val_file_list = fetch_file_list(data_dir=VALIDATION_DATA_DIR, portion=VALIDATION_PORTION)
# vg = build_numpy(file_list=val_file_list, num_samples=None, xcolumns=X_COLUMNS, ycolumns=Y_COLUMNS, ytx=None,
#                  skip_header=1, shuffle=False, is_csv=IS_CSV)

# end = timer()
print("{} min taken for generating input".format((end - start) / 60.0))
print(model_path)

checkpoint = ModelCheckpoint(new_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=5)
history = History()
tensor_board = TensorBoard(log_dir, histogram_freq=1, profile_batch=0)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=2, min_lr=1e-5)
callbacks_list = [history, tensor_board, checkpoint, reduce_lr]
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=5)

callbacks_list = [history, tensor_board, reduce_lr, checkpoint]



# hack!!!
print(K.eval(model.optimizer.lr))
adam_wn = Adam(lr=0.25e-3)
model.compile(loss="sparse_categorical_crossentropy", optimizer=adam_wn, metrics=['accuracy'])
print(K.eval(model.optimizer.lr))

model.fit(x=tg[0], y=tg[1], validation_data=vg, batch_size=BATCH_SIZE,
              epochs=EPOCHS, verbose=1, shuffle=False,
              callbacks=callbacks_list, class_weight={0: 1.0, 1: 1.0})

model.save(new_path)

5.2575216007729374e-05 min taken for generating input
/Users/lucindazhao/strava/ml-local/snapshots/fc_noWeight_6layer_l2=0.001_lr=0.001_20200219-144134
0.00025
0.00025
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 1925165 samples, validate on 212247 samples
Epoch 1/30
1925165/1925165 [==============================] - 60s 31us/sample - loss: 0.2061 - accuracy: 0.9315 - val_loss: 0.2099 - val_accuracy: 0.9281
Epoch 2/30
1925165/1925165 [==============================] - 60s 31us/sample - loss: 0.2045 - accuracy: 0.9319 - val_loss: 0.2083 - val_accuracy: 0.9285
Epoch 3/30
1925165/1925165 [==============================] - 57s 29us/sample - loss: 0.2044 - accuracy: 0.9318 - val_loss: 0.2085 - val_accuracy: 0.9282
Epoch 4/30
1925165/1925165 [==============================] - 57s 30us/sample - loss: 0.2044 - accuracy: 0.9320 - val_loss: 0.2074 - val_accuracy: 0.9284
Epoch 5/30
1924352/1925165 [============================>.] - ETA: 0s - loss: 0.2041 - accuracy: 0.9321
Epoch 000

# Score Test Data

In [2]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from timeit import default_timer as timer
from src.train.utils import build_numpy_pn, build_generator_pn, fetch_file_list

TEST_DATA = '/Users/lucindazhao/strava/ml-local/testData/set1/'

start = timer()
test_file_list = fetch_file_list(data_dir=TEST_DATA, portion=2.1)

X, y = build_numpy(file_list=test_file_list, num_samples=None, xcolumns=X_COLUMNS,
                   ycolumns=Y_COLUMNS, ytx=None, is_csv=IS_CSV)
end = timer()
print("{} min taken for generating input".format((end - start) / 60.0))


def model_test(model_path, X, y):
    model = load_model(model_path)
    print(model.summary())
    print(model.metrics_names)
    print(K.eval(model.optimizer.lr))
    y_hat = model.predict(X)
    return y_hat, y


Number of Files: 2
0.070376587366142 min taken for generating input


In [12]:
model_path = '/Users/lucindazhao/strava/ml-local/snapshots/fc_noWeight_dropOut_l2=0.001_lr=0.001_20200219-122705_resume_20200219-124048.h5'

y_hat, y = model_test(model_path, X, y)

Model: "model_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_56 (InputLayer)        [(None, 69)]              0         
_________________________________________________________________
dense1 (Dense)               (None, 64)                4480      
_________________________________________________________________
batch_normalization_124 (Bat (None, 64)                256       
_________________________________________________________________
dropout_50 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense2 (Dense)               (None, 128)               8320      
_________________________________________________________________
batch_normalization_125 (Bat (None, 128)               512       
_________________________________________________________________
dropout_51 (Dropout)         (None, 128)               0  

In [13]:
y_true = y.reshape(-1)
y_scores = y_hat[:, 1].reshape(-1)

result = np.stack([y_true, y_scores], axis=-1)



In [14]:
print(model_path)

pieces = model_path.split('/')
model_id = pieces[-1]
print(model_id)

/Users/lucindazhao/strava/ml-local/snapshots/fc_noWeight_dropOut_l2=0.001_lr=0.001_20200219-122705_resume_20200219-124048.h5
fc_noWeight_dropOut_l2=0.001_lr=0.001_20200219-122705_resume_20200219-124048.h5


In [15]:
np.savetxt("./result/{}.csv".format(model_id), result, fmt=['%1.0f', '%1.3f'],
           delimiter=',', newline='\n', header='label,score')

# Model Format

In [2]:
h5_model = load_model('/Users/lucindazhao/strava/ml-local/models/V0/cnn_noWeight_lr=0.001_20200218-224657.h5')


In [5]:
h5_model.save('/Users/lucindazhao/strava/ml-local/models/V0/cnn_noWeight_lr=0.001_20200218-224657')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /Users/lucindazhao/strava/ml-local/models/V0/cnn_noWeight_lr=0.001_20200218-224657/assets


In [7]:
h5_model.summary()

Model: "model_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_48 (InputLayer)        [(None, 69)]              0         
_________________________________________________________________
reshape_22 (Reshape)         (None, 69, 1)             0         
_________________________________________________________________
conv0 (Conv1D)               (None, 69, 32)            128       
_________________________________________________________________
batch0 (BatchNormalization)  (None, 69, 32)            128       
_________________________________________________________________
dropout_32 (Dropout)         (None, 69, 32)            0         
_________________________________________________________________
conv1 (Conv1D)               (None, 69, 32)            3104      
_________________________________________________________________
batch1 (BatchNormalization)  (None, 69, 32)            128

In [18]:
saved_model = load_model('/Users/lucindazhao/strava/ml-local/models/V0/cnn_noWeight_lr=0.001_20200218-224657')

In [20]:
saved_model.summary()

Model: "model_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_48 (InputLayer)        [(None, 69)]              0         
_________________________________________________________________
reshape_22 (Reshape)         (None, 69, 1)             0         
_________________________________________________________________
conv0 (Conv1D)               (None, 69, 32)            128       
_________________________________________________________________
batch0 (BatchNormalization)  (None, 69, 32)            128       
_________________________________________________________________
dropout_32 (Dropout)         (None, 69, 32)            0         
_________________________________________________________________
conv1 (Conv1D)               (None, 69, 32)            3104      
_________________________________________________________________
batch1 (BatchNormalization)  (None, 69, 32)            128